# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
pip install citipy

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys2 import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
weather_df =pd.read_csv('../WeatherPy/citiesdata.csv')
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,busselton,-33.6500,115.3333,62.01,73,92,1.99,AU,2021-04-30 01:38:26
1,iracoubo,5.4802,-53.2011,76.28,87,100,7.11,GF,2021-04-30 01:38:26
2,puerto ayora,-0.7393,-90.3518,74.68,84,85,7.25,EC,2021-04-30 01:38:26
3,tottori,35.5000,134.2333,64.40,59,75,5.75,JP,2021-04-30 01:38:27
4,east london,-33.0153,27.9116,59.00,94,20,9.22,ZA,2021-04-30 01:38:27


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)
# HINT: be sure to handle NaN values
weather_df = weather_df.dropna()
# Store 'Lat' and 'Lng' into  locations 
locations = weather_df[['Lat','Lng']].astype(float)
# Convert Humidity to float and store
weight = weather_df['Humidity'].astype(float)

In [7]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1.5)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
#A max temperature lower than 80 degrees but higher than 50.
condition_a = (weather_df['Max Temp'] < 77) & (weather_df['Max Temp'] > 30)
#Wind speed less than 12 mph.
condition_b = (weather_df['Wind Speed'] < 30)
#Zero cloudiness.
condition_c = (weather_df['Cloudiness'] > 50)
#Drop any rows that don't contain all three conditions. 
weather_condition = condition_a & condition_b & condition_c
#You want to be sure the weather is ideal.
fitting_weather = weather_df[weather_condition]
fitting_weather                   

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,busselton,-33.6500,115.3333,62.01,73,92,1.99,AU,2021-04-30 01:38:26
1,iracoubo,5.4802,-53.2011,76.28,87,100,7.11,GF,2021-04-30 01:38:26
2,puerto ayora,-0.7393,-90.3518,74.68,84,85,7.25,EC,2021-04-30 01:38:26
3,tottori,35.5000,134.2333,64.40,59,75,5.75,JP,2021-04-30 01:38:27
6,guasdualito,7.2424,-70.7324,75.20,100,100,1.14,VE,2021-04-30 01:38:28
...,...,...,...,...,...,...,...,...,...
550,colares,38.7992,-9.4469,57.00,81,100,3.00,PT,2021-04-30 01:42:35
554,vilcun,-38.6500,-72.2333,51.80,100,90,2.30,CL,2021-04-30 01:42:36
556,havre-saint-pierre,50.2334,-63.5986,41.00,87,90,9.22,CA,2021-04-30 01:42:37
558,entre rios,-11.9419,-38.0844,69.10,97,56,3.67,BR,2021-04-30 01:42:38


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Store into variable named hotel_df.
hotel_df = fitting_weather

hotel_df['Hotel Name'] = ""

#Set parameters to search for hotels with 5000 meters.

hotels = hotel_df[['City','Lat','Lng']]


# create a params dict that will be updated with new city each iteration

# Loop through the cities_pd and run a lat/long search for each city
for index, row in hotels.iterrows():
        
        lat = hotels['Lat'][index]
        lng = hotels['Lng'][index]
        # update address key value
        params = {
            'location':f'{lat},{lng}',
            'rankby': 'distance',
            'keyword':'Hotel',
            'radious': 5000,
            'key':g_key,}
        
        # make request
        response = requests.get(base_url, params=params)
        response = response.json()
        
        try:
            hotels.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
            hotels.loc[index, "Rating"] = name_address["results"][0]["rating"]
            htls.append(results[0]["name"])
        except:
            print('Missing field/result... skipping.')
    

<ipython-input-12-baefc9528ede>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [10]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
# Display figure
fig


Figure(layout=FigureLayout(height='420px'))